In [1]:
import pandas as pd
import numpy as np
from time import time

from numpy.random import seed
import tensorflow as tf
from scipy.spatial.distance import cosine

In [2]:
# los archivos grandes estan comentados para que lea por defecto los pequeños y no tarde mil años

# ruta relativa de los embeddings
path = '../data/KDD2023_embeddings/'

# Archivos de muestra
train_inputs_filename  =  "train_inputs_produ_long_2_DE_embeddings_100_muestra5000bloques.csv"
train_outputs_filename = "train_outputs_produ_long_2_DE_embeddings_100_muestra5000bloques.csv"
test_inputs_filename   =   "test_inputs_produ_long_2_DE_embeddings_100_muestra5000bloques.csv"
test_outputs_filename  =  "test_outputs_produ_long_2_DE_embeddings_100_muestra5000bloques.csv"


# Archivos grandes
#train_inputs_filename  =  "train_inputs_produ_long_2_DE_embeddings_100.csv"
#train_outputs_filename = "train_outputs_produ_long_2_DE_embeddings_100.csv"
#test_inputs_filename   =   "test_inputs_produ_long_2_DE_embeddings_100.csv"
#test_outputs_filename  = " test_outputs_produ_long_2_DE_embeddings_100.csv"

# Embedding de todos los items
items_filename = "Primeros_embeddings.csv"

In [3]:
# Train inputs
t1 = time()
chunk1 = pd.read_csv(path+train_inputs_filename, 
                    encoding = 'unicode_escape', engine ='python',chunksize=1e5)
print(f"Tiempo chunck1:\t{time()-t1}")
train_inputs_df = pd.concat(chunk1)
print(f"Tiempo total1:\t{(time()-t1)/60} min\n")


# Train outputs
t2 = time()
chunk2 = pd.read_csv(path+train_outputs_filename,
                    encoding = 'unicode_escape', engine ='python',chunksize=1e5)
print(f"Tiempo chunck2:\t{time()-t2}")
train_outputs_df = pd.concat(chunk2)
print(f"Tiempo total2:\t{(time()-t2)/60} min\n")


# Test inputs
t3 = time()
chunk3 = pd.read_csv(path+test_inputs_filename, 
                    encoding = 'unicode_escape', engine ='python',chunksize=1e5)
print(f"Tiempo chunck3:\t{time()-t3}")
test_inputs_df = pd.concat(chunk3)
print(f"Tiempo total3:\t{(time()-t3)/60} min\n")


# Test ouptuts
t4 = time()
chunk4 = pd.read_csv(path+test_outputs_filename, 
                    encoding = 'unicode_escape', engine ='python',chunksize=1e5)
print(f"Tiempo chunck4:\t{time()-t4}")
test_outputs_df = pd.concat(chunk4)
print(f"Tiempo total4:\t{(time()-t4)/60} min\n")


# Items 
t5 = time()
chunk5 = pd.read_csv(path+items_filename, 
                    encoding = 'unicode_escape', engine ='python',chunksize=1e5)                     
print(f"Tiempo chunck5:\t{time()-t5}")
items_df = pd.concat(chunk5)
print(f"Tiempo total5:\t{(time()-t5)/60} min\n")

Tiempo chunck1:	0.0
Tiempo total1:	0.019404645760854086 min

Tiempo chunck2:	0.0
Tiempo total2:	0.009508176644643148 min

Tiempo chunck3:	0.0
Tiempo total3:	0.01869349479675293 min

Tiempo chunck4:	0.0039937496185302734
Tiempo total4:	0.010634005069732666 min

Tiempo chunck5:	0.0
Tiempo total5:	0.9642778873443604 min



In [4]:
train_inputs_df.head(42)

,Time,SessionId,price,_0,_1,_2,_3,_4,_5,_6,...,_91,_92,_93,_94,_95,_96,_97,_98,_99,bloque
0,0,1,39.80,0.664977,-0.069365,0.506529,0.313821,0.436912,0.707025,-0.555230,...,0.265722,0.227143,-0.038207,0.603852,0.576412,-0.392992,1.065884,-0.131236,0.103233,1
1,1,1,40.27,0.570443,0.488517,-0.553709,-0.796355,0.011448,-0.705797,0.165171,...,0.435880,0.221385,-0.056664,0.074340,-0.728135,0.642762,-0.831886,-0.091667,-0.627620,1
2,0,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,2
3,1,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,2
4,1,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,3
5,2,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,3
6,2,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,4
7,3,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,4
8,3,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,5
9,4,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,5


In [5]:
train_outputs_df.head(42)

,Time,SessionId,price,_0,_1,_2,_3,_4,_5,_6,...,_91,_92,_93,_94,_95,_96,_97,_98,_99,bloque
0,2,1,29.99,-0.683102,-0.572628,0.081564,0.351446,-0.757660,0.195354,0.251838,...,1.087058,-0.072720,0.336350,0.310282,1.152645,0.643065,0.360376,-0.534070,-0.181189,1
1,2,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,2
2,3,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,3
3,4,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,4
4,5,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,5
5,6,2,51.99,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,...,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708,6
6,7,2,67.98,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,...,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181,7
7,2,3,14.99,0.143008,0.875333,0.994746,-1.709153,0.714983,-0.365896,0.429587,...,-0.172792,-0.050489,0.172538,1.036282,0.419928,-1.668528,-0.235691,-0.007240,0.286061,8
8,3,3,13.99,0.150923,1.053898,0.294180,-1.188020,1.136249,-0.479739,0.803053,...,-0.437277,-0.064005,0.186920,0.563317,0.589618,-1.533040,0.253175,-0.225902,0.247341,9
9,4,3,13.99,0.091262,0.985513,0.502738,-1.624523,0.951381,-0.703418,0.556887,...,-0.242920,-0.207686,0.267576,0.551609,0.566430,-1.905633,0.360721,0.059032,-0.025009,10


In [6]:
test_inputs_df.head(42)

,SessionId,ItemId,price,Time,bloque,_0,_1,_2,_3,_4,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,4,B09Y5CSL3T,35.99,0,1,0.129319,0.539892,-0.032077,-0.256253,0.011991,...,0.041062,-1.203274,-0.476189,0.722165,0.215479,0.567653,0.338178,0.363648,-0.725252,-0.137194
1,4,B09Y5DPTXN,21.99,1,1,0.224866,0.153675,0.012376,-1.131105,0.859782,...,-0.248609,-1.111776,-0.914670,-0.490817,0.061933,0.251597,-0.501919,-0.429823,-0.318810,0.570877
2,5,B0749V8TC7,16.64,0,2,-0.272628,0.477413,-0.465077,-0.982886,0.346243,...,0.594582,-0.827186,-0.258304,-0.006086,0.805035,-0.474649,0.491005,-0.079876,0.361759,0.388552
3,5,B0749W93VC,23.00,1,2,-0.006274,-0.577533,-0.423515,-0.854287,-0.417243,...,-0.414666,0.908716,-0.207442,-0.020046,0.501758,0.389630,0.258546,0.554258,-0.217217,0.067072
4,7,B09B2W5S9R,17.84,0,3,0.233968,-0.072843,0.525812,-0.164559,-0.457568,...,-0.264902,0.306018,-0.076854,-0.741276,1.305011,-0.092469,-0.530867,-0.008583,0.492039,-1.062940
5,7,B09B2YFY6M,18.98,1,3,0.359259,0.111653,0.386228,0.279619,-0.516023,...,0.225225,-0.052221,-0.656121,0.155096,-0.122652,0.226232,0.201553,-0.075336,-0.484820,-0.524206
6,8,B01MQOR80Q,14.99,0,4,-0.314846,1.045331,-0.578234,0.857564,0.366818,...,1.008817,0.294121,0.194643,0.399610,-0.801023,0.421279,-0.150969,0.223606,-0.905509,-0.084606
7,8,B095HS8R62,11.99,1,4,0.215774,-0.119771,0.974615,0.706242,-0.313189,...,0.058480,-0.198108,0.264937,-0.745676,-0.544075,0.450095,0.295693,-0.355768,0.827401,0.635367
8,18,B00CS3IXLA,94.90,0,5,0.715521,-0.316147,-0.274828,-0.270777,0.596134,...,-0.234783,0.089968,0.210184,0.550276,0.635939,-0.904226,0.387844,-0.747041,-0.043086,0.073935
9,18,B0751FNR26,89.90,1,5,1.113170,-0.286217,-0.454359,-0.215475,-0.732388,...,-0.580249,-0.541340,-1.415283,-0.346398,-1.243809,0.671391,-0.352190,0.797040,-0.340970,-0.537909


In [7]:
test_outputs_df.head(42)

,SessionId,ItemId,price,Time,bloque,_0,_1,_2,_3,_4,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,4,B09FKD61R8,12.99,2,1,-0.002945,-0.309385,-0.234677,-0.606455,-0.269712,...,-0.222802,-0.716954,-0.405447,-0.101807,-0.121946,0.207062,-0.089704,0.219215,-0.174642,0.032829
1,5,B0749TX4YP,16.94,2,2,-0.149813,0.923789,0.213354,0.098561,-0.017821,...,0.750175,-1.016626,-0.548267,-0.471347,0.237880,-0.343647,-0.026971,0.365171,0.619913,-0.650842
2,7,B09B2WGPRB,25.26,2,3,0.730009,0.026270,-0.196405,-0.471839,0.616129,...,-0.583010,-0.051771,0.282293,0.734268,0.218641,-0.442241,-0.646228,0.717754,-0.044733,0.053107
3,8,B09B31WTVY,14.99,2,4,-0.546903,0.399856,-0.210085,0.739138,0.391940,...,0.438744,0.039682,0.381522,-0.684942,-0.196225,0.838859,0.331715,-0.401395,-0.227708,0.494975
4,18,B000OXME9S,15.99,2,5,0.813711,0.679625,0.080360,-1.881615,-0.025484,...,0.936305,0.770865,0.086103,0.329321,0.463898,-0.483085,0.272671,-0.131527,-0.574167,0.443757
5,24,B00E4ZS2M0,1.43,2,6,1.375048,-0.081576,0.402155,-0.252120,0.251578,...,-0.105165,-0.805530,0.682669,-0.070213,-0.055287,0.211441,-1.106829,-0.271729,0.077695,0.147439
6,25,B09DR42FJG,20.95,2,7,0.362070,0.086902,-0.129951,0.600461,-0.268295,...,0.822509,0.212139,0.087502,0.009151,-1.338473,-0.380201,-0.716978,-0.194428,-0.528416,0.211798
7,32,B073FGSC7T,67.47,2,8,-0.127130,-0.009549,-0.596064,-1.052076,0.393356,...,-0.437052,0.124138,-0.110222,-0.223278,0.267996,-0.594351,0.085760,-0.867766,-0.132497,-0.289764
8,33,B097Y4B9N8,58.90,2,9,-0.204002,0.011629,0.524546,-0.251743,-0.230956,...,0.107486,-0.035796,-0.207202,-0.579401,0.544332,0.528831,-0.106028,0.535536,0.141423,-0.249508
9,41,B01CO5PIWS,6.98,2,10,1.105012,-0.364182,-0.184307,0.533026,0.357683,...,-0.161971,-0.541462,-0.219024,-0.515478,0.210240,-0.188434,-0.537253,-0.424261,-0.333918,0.429896


In [8]:
# Aquí solo cojo el embedding, que es lo que quiero predecir
# Poner hasta la 104 para coger el PRICE
train_inputs_embedding_df = train_inputs_df.iloc[:,3:103]
train_inputs_embedding_df.head(42)

,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,0.664977,-0.069365,0.506529,0.313821,0.436912,0.707025,-0.555230,0.645436,-0.946387,0.014305,...,0.473874,0.265722,0.227143,-0.038207,0.603852,0.576412,-0.392992,1.065884,-0.131236,0.103233
1,0.570443,0.488517,-0.553709,-0.796355,0.011448,-0.705797,0.165171,1.431727,-0.057081,-0.746036,...,-0.053404,0.435880,0.221385,-0.056664,0.074340,-0.728135,0.642762,-0.831886,-0.091667,-0.627620
2,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
3,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
4,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
5,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
6,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
7,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
8,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
9,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181


In [9]:
# Aquí solo cojo el embedding, que es lo que quiero predecir, el PRICE en ningún caso (creo)
train_outputs_embedding_df = train_outputs_df.iloc[:,3:103]
train_outputs_embedding_df.head(42)

,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,-0.683102,-0.572628,0.081564,0.351446,-0.757660,0.195354,0.251838,-0.778700,0.244938,-0.180675,...,0.403047,1.087058,-0.072720,0.336350,0.310282,1.152645,0.643065,0.360376,-0.534070,-0.181189
1,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
2,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
3,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
4,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
5,0.711450,-0.594349,-0.571091,0.135952,-0.176675,0.902084,-0.421772,0.979994,-0.553360,0.027897,...,-0.619394,-0.630809,-0.629734,0.344837,0.495974,-0.057006,0.983843,-0.392710,-0.052978,-0.258708
6,-0.130319,-0.749803,0.816655,0.910202,-0.515009,0.834889,-0.259825,-0.539267,-0.491490,-0.324758,...,-0.021406,-0.002050,0.167245,0.475689,0.229178,0.853722,0.595040,0.168473,0.212663,0.681181
7,0.143008,0.875333,0.994746,-1.709153,0.714983,-0.365896,0.429587,-0.582266,-0.335266,0.607200,...,-0.615918,-0.172792,-0.050489,0.172538,1.036282,0.419928,-1.668528,-0.235691,-0.007240,0.286061
8,0.150923,1.053898,0.294180,-1.188020,1.136249,-0.479739,0.803053,-0.079197,-0.686259,0.694010,...,-0.078176,-0.437277,-0.064005,0.186920,0.563317,0.589618,-1.533040,0.253175,-0.225902,0.247341
9,0.091262,0.985513,0.502738,-1.624523,0.951381,-0.703418,0.556887,0.087097,-0.295441,0.536528,...,-0.591025,-0.242920,-0.207686,0.267576,0.551609,0.566430,-1.905633,0.360721,0.059032,-0.025009


In [10]:
window_len = 2

LSTM_train_inputs_df = []

for i in range(int(len(train_inputs_embedding_df)/2)):
    temp_set = train_inputs_embedding_df[(i*window_len):((i+1)*window_len)].copy()
    LSTM_train_inputs_df.append(temp_set)
    
print(len(LSTM_train_inputs_df))

5000


In [11]:
LSTM_train_outputs_df = []

for i in range(int(len(train_outputs_embedding_df))):
    temp_set = train_outputs_embedding_df[i:i+1].copy()
    LSTM_train_outputs_df.append(temp_set)
    
print(len(LSTM_train_outputs_df))

5000


In [12]:
print("LSTM_train_inputs_df[0]")
print(LSTM_train_inputs_df[0])
print("LSTM_train_inputs_df[1]")
print(LSTM_train_inputs_df[1])
print("LSTM_train_inputs_df[2]")
print(LSTM_train_inputs_df[2])
print("LSTM_train_outputs_Df[0]")
print(LSTM_train_outputs_df[0])
print("LSTM_train_outputs_Df[1]")
print(LSTM_train_outputs_df[1])
print("LSTM_train_outputs_Df[2]")
print(LSTM_train_outputs_df[2])

LSTM_train_inputs_df[0]
         _0        _1        _2        _3        _4        _5        _6  \
0  0.664977 -0.069365  0.506529  0.313821  0.436912  0.707025 -0.555230   
1  0.570443  0.488517 -0.553709 -0.796355  0.011448 -0.705797  0.165171   

         _7        _8        _9  ...       _90       _91       _92       _93  \
0  0.645436 -0.946387  0.014305  ...  0.473874  0.265722  0.227143 -0.038207   
1  1.431727 -0.057081 -0.746036  ... -0.053404  0.435880  0.221385 -0.056664   

        _94       _95       _96       _97       _98       _99  
0  0.603852  0.576412 -0.392992  1.065884 -0.131236  0.103233  
1  0.074340 -0.728135  0.642762 -0.831886 -0.091667 -0.627620  

[2 rows x 100 columns]
LSTM_train_inputs_df[1]
         _0        _1        _2        _3        _4        _5        _6  \
2 -0.130319 -0.749803  0.816655  0.910202 -0.515009  0.834889 -0.259825   
3  0.711450 -0.594349 -0.571091  0.135952 -0.176675  0.902084 -0.421772   

         _7       _8        _9  ...       _

In [13]:
import numpy as np

# Convertimos los datos en arrays para que puedan ser procesados por la LSTM

X = [np.array(dataframe) for dataframe in LSTM_train_inputs_df]
X = np.array(X)

Y = [np.array(dataframe) for dataframe in LSTM_train_outputs_df]
Y = np.array(Y)

In [15]:
print(X.shape)
print(Y.shape)

(5000, 2, 100)
(5000, 1, 100)


In [16]:
# La idea es hacer corresponder a cada observación de los bloques de X, la siguiente observación (en Y)
# Esto permite usar la opción return_sequence = True y, a través de ella, un entrenamiento mejor
nueva_Y = np.zeros((5000,2,100))
# Para fabricar la nueva Y, se elimina la primera observación de la X de cada bloque
# Las n-1 primeras observaciones de Y van a ser las n-1 últimas de X
nueva_Y[:,0:1,:] = X[:,1:2,:]
# Para fabricar la n-ésima observación de la nueva Y, se utiliza la que se tenía
nueva_Y[:,1:2,:] = Y
print(nueva_Y[0:5])

[[[ 5.70442860e-01  4.88516800e-01 -5.53709450e-01 -7.96355100e-01
    1.14475570e-02 -7.05797000e-01  1.65171100e-01  1.43172740e+00
   -5.70806970e-02 -7.46035700e-01 -7.15257400e-01  7.05055300e-02
   -4.22730830e-01  1.95171790e-03 -4.10601380e-01  1.74761240e-01
   -1.44318770e-01  7.76106500e-01  3.83912950e-01 -2.67384800e-01
    4.60527300e-02 -3.75916330e-01  4.64138600e-01 -4.73758850e-01
    1.25645340e-01  2.84985720e-01  3.59824400e-01  8.65332100e-02
    7.92713760e-01 -9.52782400e-02 -3.07462160e-01  1.70408100e-01
    2.79219660e-01  4.33774620e-01 -5.59762400e-01 -1.03654710e+00
    1.16946440e-01 -1.16095890e-01  1.05050430e+00 -1.29305630e-01
   -6.49193600e-01 -2.71271300e-01  2.70173340e-01 -2.26760970e-02
    5.73202400e-01  4.24084200e-01  1.30603650e+00  4.03137480e-01
    4.20810850e-01 -8.67623700e-01 -2.49570520e-01  6.28672060e-01
    7.65826100e-02  1.41612380e-01 -3.39610160e-01 -2.28401740e-01
    7.70312600e-01  8.34518700e-01 -2.13992000e-01  5.12392800

In [17]:
# comprobacion nueva Y: el primer elemento tiene que ser el mismo que el ultimo de 
nueva_Y[0,0,:5]

array([ 0.57044286,  0.4885168 , -0.55370945, -0.7963551 ,  0.01144756])

In [18]:
X[0,1,:5]

array([ 0.57044286,  0.4885168 , -0.55370945, -0.7963551 ,  0.01144756])

### Entrenamiento del modelo

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import BatchNormalization

model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1],X.shape[2]), return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(256, return_sequences=True)) # Este return_sequences = True está posibilitado por nueva_Y
model.add(BatchNormalization())
model.add(Dense(100, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 2, 256)            365568    
                                                                 
 batch_normalization (BatchN  (None, 2, 256)           1024      
 ormalization)                                                   
                                                                 
 lstm_1 (LSTM)               (None, 2, 256)            525312    
                                                                 
 batch_normalization_1 (Batc  (None, 2, 256)           1024      
 hNormalization)                                                 
                                                                 
 dense (Dense)               (None, 2, 100)            25700     
                                                                 
Total params: 918,628
Trainable params: 917,604
Non-trai

In [20]:
from numpy.random import seed
seed(221275)
#from tensorflow import set_random_seed
#set_random_seed(2)
import tensorflow as tf
tf.random.set_seed(221275)
history = model.fit(x = X, y = nueva_Y, epochs=10, batch_size=64)

Epoch 1/10
79/79 [==============================] - 10s 44ms/step - loss: 0.5643 - mse: 0.5643
Epoch 2/10
79/79 [==============================] - 3s 44ms/step - loss: 0.3445 - mse: 0.3445
Epoch 3/10
79/79 [==============================] - 4s 44ms/step - loss: 0.3164 - mse: 0.3164
Epoch 4/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2999 - mse: 0.2999
Epoch 5/10
79/79 [==============================] - 3s 44ms/step - loss: 0.2895 - mse: 0.2895
Epoch 6/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2807 - mse: 0.2807
Epoch 7/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2738 - mse: 0.2738
Epoch 8/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2676 - mse: 0.2676
Epoch 9/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2613 - mse: 0.2613
Epoch 10/10
79/79 [==============================] - 4s 45ms/step - loss: 0.2554 - mse: 0.2554


In [21]:
# Nota: Ahora al predecir, devolverá por bloque un par de predicciones (n)
# Es preciso, quedarse con la última predicción de cada bloque

In [22]:
test_inputs_embedding_df = test_inputs_df.iloc[:,5:105]
test_inputs_embedding_df.head()

,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,0.129319,0.539892,-0.032077,-0.256253,0.011991,-0.364310,-0.367446,0.355197,0.254746,1.432837,...,0.041062,-1.203274,-0.476189,0.722165,0.215479,0.567653,0.338178,0.363648,-0.725252,-0.137194
1,0.224866,0.153675,0.012376,-1.131105,0.859782,-1.041499,0.338344,0.458968,-0.296176,-0.064848,...,-0.248609,-1.111776,-0.914670,-0.490817,0.061933,0.251597,-0.501919,-0.429823,-0.318810,0.570877
2,-0.272628,0.477413,-0.465077,-0.982886,0.346243,0.341149,-0.530111,-0.248136,-0.279904,0.936272,...,0.594582,-0.827186,-0.258304,-0.006086,0.805035,-0.474649,0.491005,-0.079876,0.361759,0.388552
3,-0.006274,-0.577533,-0.423515,-0.854287,-0.417243,0.577934,-0.614029,-1.013557,-0.100823,-0.229151,...,-0.414666,0.908716,-0.207442,-0.020046,0.501758,0.389630,0.258546,0.554258,-0.217217,0.067072
4,0.233968,-0.072843,0.525812,-0.164559,-0.457568,-0.264998,0.343169,0.102781,-0.816469,0.485059,...,-0.264902,0.306018,-0.076854,-0.741276,1.305011,-0.092469,-0.530867,-0.008583,0.492039,-1.062940


In [23]:
test_outputs_embedding_df = test_outputs_df.iloc[:,5:105]
test_outputs_embedding_df.head()

,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,...,_90,_91,_92,_93,_94,_95,_96,_97,_98,_99
0,-0.002945,-0.309385,-0.234677,-0.606455,-0.269712,0.064201,-0.555288,0.276279,-0.388699,-0.403468,...,-0.222802,-0.716954,-0.405447,-0.101807,-0.121946,0.207062,-0.089704,0.219215,-0.174642,0.032829
1,-0.149813,0.923789,0.213354,0.098561,-0.017821,-0.337957,-0.316720,0.055588,0.850836,-0.453465,...,0.750175,-1.016626,-0.548267,-0.471347,0.237880,-0.343647,-0.026971,0.365171,0.619913,-0.650842
2,0.730009,0.026270,-0.196405,-0.471839,0.616129,0.207573,0.425365,0.356260,-0.155228,-0.430035,...,-0.583010,-0.051771,0.282293,0.734268,0.218641,-0.442241,-0.646228,0.717754,-0.044733,0.053107
3,-0.546903,0.399856,-0.210085,0.739138,0.391940,-0.066967,-1.173417,-0.115633,0.168700,0.962905,...,0.438744,0.039682,0.381522,-0.684942,-0.196225,0.838859,0.331715,-0.401395,-0.227708,0.494975
4,0.813711,0.679625,0.080360,-1.881615,-0.025484,0.148472,0.762601,0.633669,-0.806410,-0.105565,...,0.936305,0.770865,0.086103,0.329321,0.463898,-0.483085,0.272671,-0.131527,-0.574167,0.443757


In [24]:
LSTM_test_inputs_df = []

for i in range(int(len(test_inputs_embedding_df)/2)):
    temp_set = test_inputs_embedding_df[(i*window_len):((i+1)*window_len)].copy()
    LSTM_test_inputs_df.append(temp_set)
    
print(len(LSTM_test_inputs_df))

5000


In [25]:
LSTM_test_outputs_df = []

for i in range(int(len(test_outputs_embedding_df))):
    temp_set = test_outputs_embedding_df[i:i+1].copy()
    LSTM_test_outputs_df.append(temp_set)
    
print(len(LSTM_test_outputs_df))

5000


In [26]:
X_test = [np.array(dataframe) for dataframe in LSTM_test_inputs_df]
X_test = np.array(X_test)

#Y_test = [np.array(dataframe) for dataframe in LSTM_test_outputs_df]
#Y_test = np.array(Y_test)

# Para que la salida tenga tamaño (5000,100) en vez de (5000,1,100)
Y_test = np.concatenate(LSTM_test_outputs_df)

In [27]:
Y_big_pred = model.predict(X_test)

157/157 [==============================] - 3s 11ms/step


In [28]:
Y_big_pred.shape

(5000, 2, 100)

In [29]:
# Comprobacion (la primera prediccion deberia ser el ultimo item de la sesion, aunque no coincide)
Y_big_pred[0,0,:10]

array([ 0.15510052,  0.1925848 ,  0.07851554, -0.04987381, -0.07452232,
        0.05541576, -0.2851217 , -0.17376794,  0.00973467,  0.3198682 ],
      dtype=float32)

In [30]:
X_test[0,1,:10]

array([ 0.22486581,  0.1536754 ,  0.01237574, -1.1311046 ,  0.8597818 ,
       -1.041499  ,  0.33834422,  0.45896804, -0.2961764 , -0.06484785])

In [31]:
# Nos quedamos con el ultimo vector para cada sesion
Y_pred = Y_big_pred[:,-1,:]
Y_pred.shape

(5000, 100)

In [32]:
#Y_muestra = Y_pred[:10,:]
#Y_muestra.shape

(10, 100)

In [33]:
#items_muestra = items_df.head(1000)
#items_muestra.head()

,ItemId,movie_id,0,1,2,3,4,5,6,7,...,90,91,92,93,94,95,96,97,98,99
0,0007440847,0,0.705512,-0.132848,0.360967,0.204053,-0.491573,0.525456,-1.061789,-0.450654,...,-0.676208,0.531552,-0.686487,0.003330,-0.311290,0.755625,0.135033,0.538100,0.209097,-0.048732
1,0007477155,1,-0.548283,0.000355,-0.857576,-0.516308,-0.164168,0.305688,0.822554,-0.304165,...,0.833657,0.844858,-0.135204,-0.634501,-0.174124,0.235411,0.106691,-0.248621,-0.488445,0.233837
2,0008260184,10,-0.648677,-0.475611,-0.211740,-0.030497,-0.084687,-0.893777,0.549404,-0.272313,...,0.181996,-0.526775,0.429677,0.377227,-0.394160,-0.711318,-0.110256,-0.051346,0.403998,0.276496
3,0441569595,100,-0.809459,0.160266,-0.462641,-1.598935,-0.229119,0.427275,0.659189,-0.290117,...,-0.787689,0.261570,0.718889,-0.159298,0.680047,-0.958712,-0.172936,-0.168796,0.013271,0.444924
4,1973363178,1000,0.711795,-0.133876,0.279420,-0.697728,-0.295069,-0.025046,-0.499776,-0.012979,...,-0.289264,0.664851,-0.508413,-0.638434,0.007214,0.349008,0.156555,-0.019364,0.082528,0.762302


In [36]:
#t = time()
#preds_lst = []
#for row in Y_muestra:
#    distancias = items_muestra.iloc[:,2:].apply(cosine,axis=1,v=row)
#    print(f"Puntuar: {(time()-t)/60} min")
#    t=time()
#    preds_lst.append(items_muestra.iloc[distancias.sort_values(ascending=True).head(100).index]['ItemId'].to_list())
#    print(f"Ordenar y pasar a lista: {(time()-t)/60} min\n")
#print(f"Tiempo total: {(time()-t)/60} min\n")

Puntuar: 0.0024000883102416994 min
Ordenar y pasar a lista: 6.666978200276693e-05 min

Puntuar: 0.0021632512410481772 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.0018228491147359212 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.0018497506777445476 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.0015624443689982097 min
Ordenar y pasar a lista: 0.00035300652186075844 min

Puntuar: 0.002154636383056641 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.0017697652180989583 min
Ordenar y pasar a lista: 9.124676386515299e-05 min

Puntuar: 0.0019179423650105794 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.0018584529558817546 min
Ordenar y pasar a lista: 0.0 min

Puntuar: 0.001858051617940267 min
Ordenar y pasar a lista: 0.0 min

Tiempo total: 0.0 min



In [38]:
# Creamos el dataset de la prediccion (ejemplo con las muestras)
predictions = pd.DataFrame()
predictions['next_item'] = test_outputs_df['ItemId'].head(len(preds_lst))
predictions['next_item_prediction'] = preds_lst
predictions.head()

,next_item,next_item_prediction
0,B09FKD61R8,"[3867310971, B01MU6O42P, B01MRCGW3C, B01MSJTHE..."
1,B0749TX4YP,"[B01MS7DUEM, B01MSJOH1C, B01MS6I0HQ, B01MSAXGH..."
2,B09B2WGPRB,"[B01MS1JBMZ, B01MSB3WJH, B01MU9NFEM, B01MUA5DU..."
3,B09B31WTVY,"[B01MRDMG1T, B01MSJVFC6, B01MU5VUXK, B01MT1G4E..."
4,B000OXME9S,"[3865590284, B01MT0IBIR, B01MTDFSEN, B01MRMCFD..."


In [ ]:
# En este no se muestran los tiempos, era para comprobaciones
preds_lst = []
for pred in Y_pred:
    distancias = items_df.iloc[:,2:].apply(cosine,axis=1,v=pred)
    preds_lst.append(items_df.iloc[distancias.sort_values(ascending=True).head(100).index]['ItemId'].to_list())

In [37]:
# Creamos el dataset de la prediccion
predictions = pd.DataFrame()
predictions['next_item'] = test_outputs_df['ItemId']
predictions['next_item_prediction'] = preds_lst
predictions.head()

ValueError: Length of values (10) does not match length of index (5000)

### Cálculo del MRR

In [39]:
mrr_list = []
for i, row in predictions.iterrows():
    relevant_item = row['next_item']
    prediction_list = row['next_item_prediction']
    try:
        rank = prediction_list.index(relevant_item) + 1  # Posición del ítem relevante en la lista de predicciones
        mrr = 1 / rank  # Calcular el MRR como el inverso de la posición del ítem relevante
    except ValueError:
        mrr = 0  # Si el ítem relevante no está en la lista de predicciones, asignar MRR 0
    mrr_list.append(mrr)

# Agregar la columna MRR al dataframe
predictions['MRR'] = mrr_list

In [40]:
predictions.head()

,next_item,next_item_prediction,MRR
0,B09FKD61R8,"[3867310971, B01MU6O42P, B01MRCGW3C, B01MSJTHE...",0
1,B0749TX4YP,"[B01MS7DUEM, B01MSJOH1C, B01MS6I0HQ, B01MSAXGH...",0
2,B09B2WGPRB,"[B01MS1JBMZ, B01MSB3WJH, B01MU9NFEM, B01MUA5DU...",0
3,B09B31WTVY,"[B01MRDMG1T, B01MSJVFC6, B01MU5VUXK, B01MT1G4E...",0
4,B000OXME9S,"[3865590284, B01MT0IBIR, B01MTDFSEN, B01MRMCFD...",0
